In [1]:
import pandas as pd
import numpy as np
import obspy
#import h5py
from obspy import UTCDateTime
import numpy as np
from obspy.clients.fdsn.client import Client
import matplotlib.pyplot as pl
from obspy.clients.fdsn.header import URL_MAPPINGS

#df_test = pd.read_csv("csvs/chunk5.csv")
df_test = pd.read_csv("csvs/chunk2.csv",low_memory=False)

df_test.set_index('trace_name',inplace= True)

In [2]:
df_test['station_id'] = df_test['network_code'] + '_' + df_test['receiver_code']

In [3]:
df_test = df_test.dropna(subset=['source_distance_km', 'source_latitude','source_longitude','station_id'])

In [4]:
df_test['time'] = pd.to_datetime(df_test['trace_start_time'], infer_datetime_format=True)
df_test['source_depth_km'] = pd.to_numeric(df_test['source_depth_km'],errors='coerce')
df_test['source_magnitude'] = pd.to_numeric(df_test['source_magnitude'],errors='coerce')

In [5]:
df_test.head(1)

,network_code,receiver_code,receiver_type,receiver_latitude,receiver_longitude,receiver_elevation_m,p_arrival_sample,p_status,p_weight,p_travel_sec,...,source_mechanism_strike_dip_rake,source_distance_deg,source_distance_km,back_azimuth_deg,snr_db,coda_end_sample,trace_start_time,trace_category,station_id,time
trace_name,,,,,,,,,,,,,,,,,,,,,
109C.TA_20060723155859_EV,TA,109C,BH,32.8889,-117.1051,150.0,700.0,manual,0.5,17.08,...,None,0.92,102.09,159.3,[56.79999924 55.40000153 47.40000153],[[2896.]],2006-07-23 15:59:00.960000,earthquake_local,TA_109C,2006-07-23 15:59:00.960


In [6]:
#stations_df = pd.read_csv('stations_duplicates_removed.csv')
stations_df = pd.read_csv('csvs/stations_definition.csv')
stations_df['station_opened'] =  pd.to_datetime(stations_df['station_opened'], infer_datetime_format=True)

In [7]:
set(stations_df['provider'])

{'BGR',
 'GEONET',
 'ICGC',
 'IRIS',
 'KOERI',
 'LMU',
 'NIEP',
 'NOA',
 'RESIF',
 'USP'}

In [8]:
def extract_waveform(client, event_record):
    wave = client.get_waveforms(network=event_record['network_code'],
                            station=event_record['receiver_code'],
                            starttime=UTCDateTime(event_record['time']),
                                endtime=UTCDateTime(event_record['time']) + 60,
                                location = "*",
                                channel = "*")
    
    #temprorarily extract only one stream
    return wave[0]

In [9]:
def find_existing_records(x):

        provider_of_waveform = stations_df[stations_df['station_id'] == x['station_id']]
        
        try:
            providers = provider_of_waveform['provider'].tolist()
            for p in providers:
                client = Client(p)

                wave = extract_waveform(client, x)

            return True

        except:
            return pd.NA
    

In [10]:


#df_test['has_record'] = df_test.apply(lambda x: find_existing_records(x),axis=1)

In [11]:
df_test_sub_subset = df_test.head(500)
df_test_sub_subset['has_record'] =df_test_sub_subset.apply(lambda x: find_existing_records(x),axis=1)

In [ ]:
#df_test.to_csv('earthquake_metadata_chunk2_existence_flag.csv')